In [1]:
import csv
from collections import defaultdict

# NOME DO ARQUIVO CSV ESPERADO NO MESMO DIRETÓRIO DO SCRIPT
NOME_ARQUIVO_CSV = "interacoes_globo.csv"
# Além do CSV proposto, o código foi testado com um segundo csv
# contendo erros de tipo e de valor para testar se o código
# não quebraria.

# Padronização Atribuída no Código:
# Para campos vazios que receberiam valores inteiros/floats,
# foi atribuído o valor "0".
# Para campos vazios que receberiam valores em string,
# foi atribuído o valor "".

def carregar_dados_de_arquivo_csv(nome_arquivo):
    """
    Carrega os dados de um arquivo CSV para uma lista de listas.
    A primeira lista retornada é o cabeçalho, e as subsequentes são as linhas de dados.
    Retorna None se o arquivo não for encontrado ou ocorrer um erro.
    """
    dados_com_cabecalho = []
    try:
        with open(nome_arquivo, mode='r', encoding='utf-8', newline='') as arquivo_csv:
            leitor_csv = csv.reader(arquivo_csv)
            # Adiciona todas as linhas (incluindo o cabeçalho) à lista
            for linha in leitor_csv:
                dados_com_cabecalho.append(linha)
              
       
        if not dados_com_cabecalho:
            print(f"Aviso: O arquivo CSV '{nome_arquivo}' está vazio.")
            return None
        return dados_com_cabecalho
    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo}' não encontrado. Certifique-se de que ele está na mesma pasta do script.")
        return None
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV '{nome_arquivo}': {e}")
        return None

    
def converter_lista_para_lista_de_dicionarios(dados_em_lista_com_cabecalho):
    lista_de_dicionarios = [
        dict(zip(dados_em_lista_com_cabecalho[0], linha)) for linha in dados_em_lista_com_cabecalho[1:]
    ]
    # Os dicionários são criados a partir da função zip que relaciona
    # os índices da lista 0, ou seja, o cabeçalho,
    # com os índices relacionados das demais listas [1:]
    return lista_de_dicionarios


def tratar_campos_inteiros(interacao_bruta, interacao_limpa):
    try:
        interacao_limpa["id_conteudo"] = int(interacao_bruta.get("id_conteudo", 0)) # .get recebe o valor da chave "id_conteudo" e transforma em inteiro.
    except (TypeError, ValueError):
        interacao_limpa["id_conteudo"] = 0 # Caso não seja possível trata-lo como inteiro, atribui valor 0.

    try:
        interacao_limpa["id_usuario"] = int(interacao_bruta.get("id_usuario", 0)) # .get recebe o valor da chave "id_usuario" e transforma em inteiro.
    except (TypeError, ValueError):
        interacao_limpa["id_usuario"] = 0 # Caso não seja possível tratar como inteiro, atribui valor 0.
    return interacao_limpa
    

def tratar_watch_duration_seconds(interacao_bruta, interacao_limpa):
    try:
        interacao_limpa["watch_duration_seconds"] = int(interacao_bruta.get("watch_duration_seconds", 0)) # Trata "watch_duration_seconds" como inteira e caso não seja possível, trata como excessão.
    except (TypeError, ValueError):
        interacao_limpa["watch_duration_seconds"] = 0
    return interacao_limpa


def seconds_to_HHmmss(seconds): # Função extra para converter o total de segundos em horas e minutos para facilitar visualização dos resultados.
    HH = seconds // 3600
    mm = (seconds % 3600) // 60
    ss = seconds % 60
    hours_min = (f"{HH}:{mm:02d}:{ss:02d}") # Estipulando Minutos e Segundos como dois dígitos, caso contrário poderia ficar apenas com uma casa.
    return hours_min


def tratar_campos_texto(interacao_bruta, interacao_limpa):
    # Apesar de haver vários campos baseados em texto, apenas o campo
    # "comment_text" necessitaria de tratamento contra espaçamento duplo,
    # já que é um input de string aberto para o usuário,
    # e por isso o uso dos métodos de string .join() + .split().
    # Os outros campos baseados em texto são pré-estipulados, ou seja,
    # o usuário provavelmente deve escolher-los diretamente de um
    # drop-down. Por isso, para facilitar o tratamento e padronizá-los,
    # foi atribuído o método de string .strip para limpar possíveis
    # espaços extras no início e fim.
    # Para evitar que a função quebre, foi utilizado o método .get.
    try:
        interacao_limpa["nome_conteudo"] = str(interacao_bruta.get("nome_conteudo", "")).strip()
    except (TypeError, ValueError):
        interacao_limpa["nome_conteudo"] = ""

    try:
        interacao_limpa["plataforma"] = str(interacao_bruta.get("plataforma", "")).strip()
    except (TypeError, ValueError):
        interacao_limpa["plataforma"] = ""

    try:
        interacao_limpa["tipo_interacao"] = str(interacao_bruta.get("tipo_interacao", "")).strip()
    except (TypeError, ValueError):
        interacao_limpa["tipo_interacao"] = ""

    try:
        comentario = str(interacao_bruta.get("comment_text", ""))
        interacao_limpa["comment_text"] = " ".join(comentario.split())
    except (TypeError, ValueError):
        interacao_limpa["comment_text"] = ""
    return interacao_limpa


def limpar_e_transformar_dados(lista_interacoes_brutas_dict):
    """
    Limpa e transforma os dados brutos das interações (lista de dicionários).
    Converte tipos de dados, trata valores ausentes e remove espaços.
    essa função deve usar as 3 funcoes acima
    Retorna a lista de interações limpas (também como lista de dicionários).
    """
    interacoes_limpas = []
    # Espera uma lista de dicionários aqui
    for interacao_bruta in lista_interacoes_brutas_dict:
        interacao_limpa = {}
        interacao_limpa = tratar_campos_inteiros(interacao_bruta, interacao_limpa)
        interacao_limpa = tratar_watch_duration_seconds(interacao_bruta, interacao_limpa)
        interacao_limpa = tratar_campos_texto(interacao_bruta, interacao_limpa)
        interacoes_limpas.append(interacao_limpa)
        # Chama as funcões anteriores que tratam os dados
        # e adiciona com append cada interacao_limpa à lista alteracoes_limpas,
        # formando uma nova lista de dicionários.     
    return interacoes_limpas


def criar_mapa_conteudos(interacoes_limpas):
    """
    Cria um dicionário que mapeia id_conteudo para nome_conteudo.
    """
    mapa = {}
    for interacao in interacoes_limpas:
        mapa[interacao["id_conteudo"]] = interacao.get("nome_conteudo", "")
    # Usa .get na chave "nome_conteudo" para relacionar o valor com
    # a chave "id_conteudo" correspondente.
    return mapa


def calcular_metricas_por_conteudo(interacoes_limpas, mapa_conteudos):
    """
    Calcula várias métricas de engajamento agrupadas por conteúdo.
    Aqui você deve varrer as interações limpas e calcular as métricas desejadas.
    Recomenda-se utilizar um loop e verificar se o conteudo ja foi incluido pelo nome ou id_conteudo
    Retorna um dicionário com as métricas calculadas.
    """
    metricas_conteudo = defaultdict(lambda: {
        'nome_conteudo': '',
        'total_interacoes_engajamento': 0, # like, share, comment, vote_bbb
        'contagem_por_tipo_interacao': defaultdict(int),
        'tempo_total_visualizacao': 0,
        'soma_watch_duration_para_media': 0,
        'contagem_watch_duration_para_media': 0,
        'media_tempo_visualizacao': 0.0,
        'comentarios': []
    })

    tipos_engajamento = {'like', 'share', 'comment', 'vote_bbb'}

    for interacao in interacoes_limpas:
        id_c = interacao["id_conteudo"]
        metricas_c = metricas_conteudo[id_c]

        if not metricas_c["nome_conteudo"]: # Preenche o nome do conteúdo uma vez
            metricas_c["nome_conteudo"] = mapa_conteudos.get(id_c, "Desconhecido")

        # Métrica 1: Total de interações de engajamento
        if interacao["tipo_interacao"] in tipos_engajamento:
            metricas_c["total_interacoes_engajamento"] += 1

        # Métrica 2: Contagem de cada tipo_interacao
        metricas_c["contagem_por_tipo_interacao"][interacao["tipo_interacao"]] += 1

        # Métrica 3: Tempo total de watch_duration_seconds
        metricas_c["tempo_total_visualizacao"] += interacao["watch_duration_seconds"]
        
        # Métrica 4: Média de watch_duration_seconds (considerar apenas watch_duration_seconds > 0)
        if interacao["watch_duration_seconds"] > 0:
            metricas_c["soma_watch_duration_para_media"] += interacao["watch_duration_seconds"]
            metricas_c["contagem_watch_duration_para_media"] += 1

        # Métrica 5: Listar todos os comentários
        if interacao["tipo_interacao"] == 'comment' and interacao["comment_text"]:
            metricas_c["comentarios"].append(interacao["comment_text"])

    # Calcular média de visualização final
    for id_c in metricas_conteudo:
        metricas_c = metricas_conteudo[id_c]
        if metricas_c["contagem_watch_duration_para_media"] > 0:
            metricas_c["media_tempo_visualizacao"] = round(
                metricas_c["soma_watch_duration_para_media"] / metricas_c["contagem_watch_duration_para_media"], 2
            )
        # Remover campos auxiliares
        if 'soma_watch_duration_para_media' in metricas_c:
            del metricas_c["soma_watch_duration_para_media"]
        if 'contagem_watch_duration_para_media' in metricas_c:
            del metricas_c["contagem_watch_duration_para_media"]
            
    return dict(metricas_conteudo) # Converter de volta para dict normal para exibição


def main():
    """
    Função principal para orquestrar a análise.
    """
    print("Iniciando Fase 1: Coleta e Estruturação Inicial de Dados de Engajamento Globo\n")

    # 1. Carregar dados do arquivo CSV para uma lista de listas
    dados_brutos_lista_de_listas = carregar_dados_de_arquivo_csv(NOME_ARQUIVO_CSV)
    
    if dados_brutos_lista_de_listas is None:
        print(f"Não foi possível carregar os dados do arquivo '{NOME_ARQUIVO_CSV}'. Encerrando.")
        return
    
    if len(dados_brutos_lista_de_listas) < 2: # Precisa de cabeçalho + pelo menos uma linha de dados
        print(f"O arquivo '{NOME_ARQUIVO_CSV}' não contém dados suficientes (cabeçalho e linhas de dados). Encerrando.")
        return

    print(f"Total de {len(dados_brutos_lista_de_listas) - 1} linhas de dados (mais cabeçalho) carregadas do CSV.\n")

    # ETAPA PARA OS ALUNOS: Converter a lista de listas para uma lista de dicionários
    # Esta etapa é crucial para que as funções subsequentes funcionem como esperado.
    interacoes_brutas_dict = converter_lista_para_lista_de_dicionarios(dados_brutos_lista_de_listas)
    
    
    # 2. Limpar e transformar dados (agora a partir da lista de dicionários)
    interacoes_limpas = limpar_e_transformar_dados(interacoes_brutas_dict)
    
    
    # 3. Criar mapa de conteúdos (id_conteudo -> nome_conteudo)
    mapa_conteudos = criar_mapa_conteudos(interacoes_limpas)
 

    # 4. Calcular métricas por conteúdo
    metricas = calcular_metricas_por_conteudo(interacoes_limpas, mapa_conteudos)

   
    # 5. Exibir resultados
    print("RESULTADOS:") # Utilizando estruturas fornecidos pelo código inicial
    for id_c in metricas:
        m = metricas[id_c]
        hours_min = seconds_to_HHmmss(m['tempo_total_visualizacao']) # Para facilitar visualização de resultados
        hours_media = seconds_to_HHmmss(int(m["media_tempo_visualizacao"])) # Para facilitar visualização de resultados
        total_inter = sum(m['contagem_por_tipo_interacao'].values()) # Somatória de todas interações, incluindo "view_start"
        comentarios = m["comentarios"]
        print(f"ID: {id_c} - {m['nome_conteudo']}") # ID e Nome do Conteúdo
        print(f"Total de interações: {total_inter}") # Número Total de Interações
        print("Interações por tipo:") # Quantidade de Cada Tipo de Interação
        for tipo, count in dict(m['contagem_por_tipo_interacao']).items():
            print(f"             {tipo}: {count}") # Identação Artificial Apenas por Estética
        print(f"Tempo total assistido: {m['tempo_total_visualizacao']} segundos ou {hours_min}") # Tempo Total Assistido em Segundos e Convertidos em Horas:Minutos:Segundos
        print(f"Média de tempo assistido: {m['media_tempo_visualizacao']} segundos ou {hours_media}") # Tempo Médio Assistido em Segundos e Convertidos em Horas:Minutos:Segundos
        print(f"Quantidade de comentários: {len(m['comentarios'])}") # Quantidade de Comentários por ID
        for contador, i in enumerate(comentarios, 1): # Output de Todos os Comentários em cada ID
            print(f"Comentário {contador}: {i}")
        print(f"\n")
    print("TOP 5 MAIS ASSISTIDOS:") # Lista com os 5 Conteúdos Mais Assistidos
    tempos = [(id_c, m['tempo_total_visualizacao']) for id_c, m in metricas.items()]
    tempos.sort(key=lambda x: x[1], reverse=True)

    for i in range(5):
        id_c = tempos[i][0]
        print(metricas[id_c]['nome_conteudo'])

if __name__ == "__main__":
    main()

Iniciando Fase 1: Coleta e Estruturação Inicial de Dados de Engajamento Globo

Total de 187 linhas de dados (mais cabeçalho) carregadas do CSV.

RESULTADOS:
ID: 1 - Jornal Nacional
Total de interações: 10
Interações por tipo:
             view_start: 10
Tempo total assistido: 18120 segundos ou 5:02:00
Média de tempo assistido: 1812.0 segundos ou 0:30:12
Quantidade de comentários: 0


ID: 2 - Novela Renascer
Total de interações: 10
Interações por tipo:
             view_start: 6
             like: 1
             comment: 1
             share: 2
Tempo total assistido: 16290 segundos ou 4:31:30
Média de tempo assistido: 2715.0 segundos ou 0:45:15
Quantidade de comentários: 1
Comentário 1: Que capítulo emocionante!


ID: 3 - Podcast Papo de Segunda
Total de interações: 21
Interações por tipo:
             view_start: 7
             comment: 9
             share: 3
             like: 2
Tempo total assistido: 24850 segundos ou 6:54:10
Média de tempo assistido: 3550.0 segundos ou 0:59:10
Quan